In [1]:
import csv
import numpy as np
import os
import pandas as pd
import requests
import time

In [2]:
asso = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/4_bi_random_walk_Cdataset_association.tsv")

In [3]:
omim_umls = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/omim_umls_mapping_association.tsv")

In [5]:
disgen = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/curated_gene_disease_associations.tsv", skiprows = 21)

In [19]:
disgen.head(1)

,diseaseId,geneId,score,geneName,description,diseaseName,sourceId,NofPmids,NofSnps
0,umls:C0000737,3440,0.12,IFNA2,"interferon, alpha 2",Abdominal Pain,CTD_human,1,0


In [18]:
disgen[disgen['diseaseId'].str.contains('C0035258')].head(1)

,diseaseId,geneId,score,geneName,description,diseaseName,sourceId,NofPmids,NofSnps
16282,umls:C0035258,100188812,0.120543,RLS3,Restless legs syndrome 3,Restless Legs Syndrome,CTD_human,2,0


In [20]:
omim_umls.head(1)

,OMIM ID,OMIM disease name,UMLS concept ID,UMLS concept name
0,102100,"Acromegaloid Changes, Cutis Verticis Gyrata, A...",C1868756,Corneal leukoma


In [36]:
result = pd.DataFrame()
for index, row in omim_umls.iterrows():
    umls = row['UMLS concept ID']
    found = disgen[disgen['diseaseId'].str.contains(umls)]
    if found.empty :
        continue
    else:
        for index2, row2 in found.iterrows():
            temp_df = pd.DataFrame(data = [[row['OMIM ID'], row['OMIM disease name'], umls, row['UMLS concept name'], row2['geneId'], row2['score'], row2['geneName'], row2['sourceId']]], 
                                       columns = ['OMIM_ID', 'OMIM_disease_name', 'UMLS_concept_ID', 'UMLS_concept_name', 'DisGeNet_geneId', 'DIsGeNet_score', 'DIsGeNet_geneName', 'DIsGeNet_sourceId'])
            result = result.append(temp_df) 

In [38]:
len(result['OMIM_ID'].drop_duplicates())

257

In [42]:
result.head(1)

,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,DisGeNet_geneId,DIsGeNet_score,DIsGeNet_geneName,DIsGeNet_sourceId
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188812,0.120543,RLS3,CTD_human


In [ ]:
association 364개 중에 257개 가 얼마나 해당하는지 찾아보기

In [39]:
filterd_asso = pd.read_table("/home/share/aschoi/data/Drug_Repositioning/7_new_association/5_filtered_association(drug).tsv")

In [68]:
print len(filterd_asso)
filterd_asso.head(1)

1947


,disease_id,drug_id
0,102100,DB00555


In [44]:
di_mimNum = filterd_asso['disease_id'].drop_duplicates().str[1:]
int_di_mimNum = list()
for i in di_mimNum:
    int_di_mimNum.append (int(i))
print len(int_di_mimNum)
filtered = result[result['OMIM_ID'].isin(int_di_mimNum)]
print len(filtered)
print len(filtered.drop_duplicates(subset = 'OMIM_ID'))
filtered.head()

364
10015
235


,OMIM_ID,OMIM_disease_name,UMLS_concept_ID,UMLS_concept_name,DisGeNet_geneId,DIsGeNet_score,DIsGeNet_geneName,DIsGeNet_sourceId
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188812,0.120543,RLS3,CTD_human
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188813,0.120271,RLS4,CTD_human
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100188839,0.120271,RLS5,CTD_human
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100302512,0.120271,RLS6,CTD_human
0,102300,"Restless Legs Syndrome, Susceptibility To, 1; ...",C0035258,Restless Legs Syndrome,100302561,0.120000,RLS7,CTD_human


In [77]:
len(filtered['DisGeNet_geneId'].drop_duplicates())

3521

In [ ]:
235개가 해당

In [ ]:
그럼 이 235개와 연관이 있는 drug의 수와 association의 수는?! --> 이게 gold standard 최종이 될것이다. 

In [71]:
gold = filterd_asso[filterd_asso['disease_id'].isin(filtered['OMIM_ID'])]

In [73]:
print len(gold)
print len(gold['disease_id'].drop_duplicates())
print len(gold['drug_id'].drop_duplicates())

1434
235
471


In [80]:
gold.to_csv("/home/share/aschoi/data/Drug_Repositioning/7_new_association/7_gold_association.tsv", sep='\t', index=False)
gold.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/7_new_association/7_gold_association.tsv", sep='\t', index=False)

In [81]:
gold.head()

,disease_id,drug_id
3,102300,DB00190
4,102300,DB00193
5,102300,DB00268
6,102300,DB00564
7,102300,DB00575


In [82]:
filtered.to_csv("/home/share/aschoi/data/Drug_Repositioning/6_new_disease/8_gold_disease_feature.tsv", sep="\t", index=False)
filtered.to_csv("/home/share/aschoi/nas/users/asolchoi/data/Drug_Repositioning/6_new_disease/8_gold_disease_feature.tsv", sep="\t", index=False)